In [1]:
from sys import path
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path.append("/Users/rachaelmcalister/warehouse-script/python/ab_weekly_tests/")
path.append("/Users/rachaelmcalister/jupyter_repo/")

In [3]:
from db_connector_user import DBConnectorUser
from db_connection_user import DBConnectionUser
import pandas as pd
from useful_functions import executeScriptsFromFile, calculate_conversion_CIs

In [4]:
from yaml_util import *

# Setup Database Connection

In [5]:
db_settings_dict = load_yaml_file('/Users/rachaelmcalister/jupyter_repo/db_settings.yml')
db_settings = db_settings_dict.get('db_settings')
DBConnectorUser(db_settings)
DBConnectionUser.setup_db_settings(db_settings)

# Import Data

In [6]:
df = executeScriptsFromFile('sql_queries/predict_signup_size_to_purchase_size', DBConnectionUser)

# Results

In [7]:
df['quantity_correct'] = df.quantity*df.correct_pred
df['return_quantity_correct'] = df.return_quantity*df.correct_pred
df['kept_quantity_correct'] = df.quantity_correct - df.return_quantity_correct

In [8]:
garments = [1, 6, 9, 12]
df = df[df.garment_category_id.isin(garments)]

In [9]:
df_agg = pd.DataFrame(df.groupby(['garment_category_id']).agg({'garment_category_id':'size', 'quantity': 'sum',
                                         'return_quantity': 'sum', 'correct_pred': 'sum',
                                        'quantity_correct': 'sum', 'return_quantity_correct': 'sum',
                                                              'kept_quantity_correct': 'sum'}).rename(columns={
    'garment_category_id': 'num_entries',
    'correct_pred': 'num_entries_correct',
})).reset_index()

In [10]:
# Include baseline % kept for comparison
df_agg['pct_quantity_correct_kept'] = (df_agg.kept_quantity_correct)/df_agg.quantity_correct
df_agg['pct_quantity_kept'] = (df_agg.quantity-df_agg.return_quantity)/df_agg.quantity


In [11]:
# Calculate 95% confidence intervals
df_agg['quantity_correct_kept_CI'] = df_agg.apply(lambda x: calculate_conversion_CIs(x.pct_quantity_correct_kept, x.quantity_correct), axis=1)
df_agg['quantity_kept_CI'] = df_agg.apply(lambda x: calculate_conversion_CIs(x.pct_quantity_kept, x.quantity), axis=1)
df_agg.set_index('garment_category_id', inplace=True)


In [12]:
df_agg[df_agg.columns[:7]]


,num_entries,quantity,return_quantity,num_entries_correct,quantity_correct,return_quantity_correct,kept_quantity_correct
garment_category_id,,,,,,,
1,2189,2191,1396,943.0,943,582,361
6,1904,1921,862,865.0,871,406,465
9,502,505,267,143.0,145,67,78
12,1288,1290,584,396.0,397,181,216


In [13]:
df_agg[df_agg.columns[7:]]

,pct_quantity_correct_kept,pct_quantity_kept,quantity_correct_kept_CI,quantity_kept_CI
garment_category_id,,,,
1,0.382821,0.362848,"[0.352, 0.414]","[0.343, 0.383]"
6,0.533869,0.551275,"[0.501, 0.567]","[0.529, 0.574]"
9,0.537931,0.471287,"[0.457, 0.619]","[0.428, 0.515]"
12,0.544081,0.547287,"[0.495, 0.593]","[0.52, 0.574]"
